# **Import the Necessary Libraries**

In [1]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
from stable_baselines3 import PPO, DQN, A2C, TD3, DDPG, SAC

# **Generation of the City Layout**

In [2]:
"""

Arguments: Number of rows and columns in the layout and the coordinates of an entity (possibly the Drone or the Target)
Functionality: We do not want the entity to lie on the peripheral wall of the layout (read the warning below).
               In other words, the x-coordinate of the entity must not match the zeroth or the last row. Likewise, for the
               y-coordinate.
Return: True (if the coordinates do not correspond to a peripheral wall location), else False

"""

def check_location(rows, columns, coordinates):
    
    x_coordinate, y_coordinate = coordinates[0], coordinates[1]

    if x_coordinate <= 0 or x_coordinate >= rows - 1:
        return False
    if y_coordinate <= 0 or y_coordinate >= columns - 1:
        return False

    return True

###############################################################################################################################

"""

Arguments: The dimensions of the layout (tuple of the form (rows, columns)), locations of the target and drone (tuple of
           the form (x coordinate, y coordinate)), and obstacle density (fraction of the total number of grid points covered
           by obstacles) [default: 0.2]
Functionality: Generation of a random city layout with the given dimensions and obstacle density. Note that the actual
               layout will be present in a square with dimensions (rows - 2 * columns - 2) as a perimeter wall will be
               present. 'D' and 'T' denote the location of the drone and target location respectively.
               Obstacles are denoted by '#'. The drone can use white spaces present in the returned grid for
               navigation.
Return: A randomly generated grid (not necessarily solvable!)
Warning: The location of the Drone and Target must not be on the periphery and must not coincide with each other! 
         Please provide the locations assuming zero-based indexing.
References:
            1. np.random.random_sample: https://numpy.org/doc/2.0/reference/random/generated/numpy.random.random_sample.html#numpy.random.random_sample
            2. np.where: https://numpy.org/doc/stable/reference/generated/numpy.where.html
            3. np.reshape: https://numpy.org/doc/stable/reference/generated/numpy.reshape.html
            4. ValueError: https://docs.python.org/3/tutorial/errors.html

"""

def generate_layout(dimensions, location_drone, location_target, obstacle_density = 0.2):
    
    rows, columns = dimensions[0], dimensions[1] # Extract the number of rows and columns given the dimensions
    layout = np.ones((rows, columns)) # Initiliaze the layout with all 1's (automatically takes care of the perimeter wall)

    # The grid is discrete! Therefore, the location of the Drone and Target are being converted to integral values by default.
    
    location_drone = (int(location_drone[0]), int(location_drone[1]))
    location_target = (int(location_target[0]), int(location_target[1]))

    ###########################################################################################################################

    # Check the Validity of the location of the drone and target

    if not check_location(rows, columns, location_drone):
        raise ValueError("Coordinates of the Drone are invalid! Read the Warning!")

    if not check_location(rows, columns, location_target):
        raise ValueError("Coordinates of the Target are invalid! Read the Warning!")

    if location_drone == location_target:
        raise ValueError("Location of both the drone and target are coinciding! Read the Warning")
    
    ###########################################################################################################################

    # Here, we denote the obstacles by 1 and the open space by 0. Conversion to string will be done later
    # Obtain (rows - 2) * (columns - 2) random values uniformly distributed between 0 and 1. If the random value is above
    # the obstacle density, set the grid point to 0, else 1. 
    
    layout_array = np.random.random_sample(size = (rows - 2) * (columns - 2))
    layout_array = np.where(layout_array < obstacle_density, 1, 0)

    ###########################################################################################################################

    layout_array = np.reshape(layout_array, ((rows - 2, columns - 2))) # Reshape to a (rows - 2) * (columns - 2) matrix
    layout[1 : rows - 1, 1 : columns - 1] = layout_array # This sub-matrix characterizes the city layout. Fill in the layout!

    ###########################################################################################################################

    string_format_layout = np.where(layout == 1, "#", " ") # If 1 is present => Obstacle, replace with '#'
    string_format_layout[location_drone[0], location_drone[1]] = 'D' # Set the specified drone location
    string_format_layout[location_target[0], location_target[1]] = 'T' # Set the specified target location

    ###########################################################################################################################
    
    return string_format_layout

###############################################################################################################################

# Test the Code here

drone_location = (1, 1) # Example drone location
target_location = (8, 8) # Example target location
layout_size = (10, 10) # Example layout size
print(generate_layout(layout_size, drone_location, target_location)) # For visualization of the layout

[['#' '#' '#' '#' '#' '#' '#' '#' '#' '#']
 ['#' 'D' ' ' ' ' ' ' ' ' ' ' ' ' ' ' '#']
 ['#' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' '#']
 ['#' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' '#']
 ['#' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' '#']
 ['#' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' '#']
 ['#' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' '#']
 ['#' ' ' ' ' ' ' ' ' '#' ' ' ' ' ' ' '#']
 ['#' ' ' ' ' ' ' '#' ' ' ' ' ' ' 'T' '#']
 ['#' '#' '#' '#' '#' '#' '#' '#' '#' '#']]


# **Is the Random Layout Solvable?**
### **Apply DFS!**

#### Since DFS implementation is not the focus of this work, we use the algorithm from the DSA Lectures taught by Professor Madhavan Mukund, whom we duly acknowledge.
#### Reference: https://www.youtube.com/playlist?list=PLZ2ps__7DhBaDccbZRgiU1sHX2gZrQ-XT

## **Setup the Adjacency List**

In [3]:
"""

Arguments: prospective layout, raster index for each state/ coordinate presented as a dictionary
Functionality: Using the function, we find the adjacency list. It is a dictionary where the keys represent the state indices
               , and values represent the neighboring indices. Keys and values must correspond to only non-obstacle states.
Return: Adjacency list

"""

def generate_adjacency_list(prospective, raster_index_dict):

    AList = {} # Initialize the adjacency list with an empty dictionary
    
    # We know that the peripheral wall is entirely comprised of only '#'. Therefore, the iterations start from 1 till rows - 1 (or)
    # columns - 1 accordingly
    
    for i in range(1, layout_size[0] - 1):
        for j in range(1, layout_size[1] - 1):
            if prospective[i][j] != '#': # Key should not be an obstacle
                AList[raster_index_dict[(i, j)]] = [] # If the key is not an obstacle, we are interested in the neighbors
                
                # Neighbor should not be an obstacle. If it is an obstacle, it prevents motion in that direction.
                # There are 4 possible neighbors.
                
                if prospective[i - 1][j] != '#':
                    AList[raster_index_dict[(i, j)]].append(raster_index_dict[(i - 1, j)])
    
                if prospective[i][j - 1] != '#':
                    AList[raster_index_dict[(i, j)]].append(raster_index_dict[(i, j - 1)])
    
                if prospective[i + 1][j] != '#':
                    AList[raster_index_dict[(i, j)]].append(raster_index_dict[(i + 1, j)])
    
                if prospective[i][j + 1] != '#':
                    AList[raster_index_dict[(i, j)]].append(raster_index_dict[(i, j + 1)])

    return AList

## **Implement DFS**

In [4]:
"""

Arguments: Adjacency list, two empty dictionaries named visited and parent that store information accordingly
Functionality: Initially, the DFS algorithm has yet to visit any states. Therefore, set visited as False. We are also not
               yet aware of how to visit a state from the starting state (Initial drone location). Therefore, set the parent
               as -1 for all the states. Raster scanning assigns indices starting from 0, implying that no state can have 
               a parent as -1. We safely conclude that our initialization does not result in any future conflict.
Return: The initialized form of dictionaries visited, and the parent

"""

def DFSInitList(AList, visited, parent):
    for i in AList.keys():
        visited[i] = False
        parent[i] = -1
    return visited, parent

###############################################################################################################################

"""

Arguments: Adjacency list and dictionaries visited, and parent
Functionality: Implementation of the DFS Algorithm in a recursive fashion
Return: None

"""

def DFSList(AList, v, visited, parent):
    visited[v] = True

    for k in AList[v]:
        if (not visited[k]):
            parent[k] = v
            DFSList(AList, k, visited, parent)

    return

## **Test the code here**

In [5]:
drone_location = (1, 1) # Example drone location
target_location = (8, 8) # Example target location
layout_size = (10, 10) # Example layout size
prospective = generate_layout(layout_size, drone_location, target_location, obstacle_density = 0.3) # Generate a prospective layout

###############################################################################################################################

"""

Argument: layout
Functionality: The below function provides the raster indexation corresponding to each state in the layout
Return: A dictionary where the coordinate (x, y) is the key and the raster index is the value

"""

def raster_scan(layout):
    raster_index_dict = {} # Initialize the dictionary
    counter = 0 # Enables raster scan indexation
    rows, columns = np.shape(layout)

    # Perform the raster scan

    for i in range(rows):
        for j in range(columns):
            raster_index_dict[(i, j)] = counter
            counter = counter + 1

    return raster_index_dict

###############################################################################################################################

raster_dict = raster_scan(prospective) # Obtain the dictionary with the raster indexation
AList = generate_adjacency_list(prospective, raster_dict) # Generate the adjacency list
(visited, parent) = ({}, {}) # Set visit and parent as empty dictionaries

visited, parent = DFSInitList(AList, visited, parent) # Populate these dictionaries given the adjacency list
DFSList(AList, raster_dict[drone_location], visited, parent) # Implement DFS recursively

###############################################################################################################################

print("Prospective Layout")
print(prospective)
print("States visitable by the DFS Algorithm")

visitable_states = prospective.copy() # Prevent the overwriting of the original matrix

for i in range(layout_size[0]):
    for j in range(layout_size[1]):
        if prospective[i, j] != 'D' and prospective[i, j] != 'T' and prospective[i, j] != '#':

            # Check if any state is visitable, given that it is not the start, end, or obstacle
            
            if visited[raster_dict[(i, j)]]:
                visitable_states[i, j] = 'v'

print(visitable_states)
###############################################################################################################################

if not visited[raster_dict[target_location]]:
    print("The prospective layout is not solvable")
else:
    print("The prospective layout is solvable")

Prospective Layout
[['#' '#' '#' '#' '#' '#' '#' '#' '#' '#']
 ['#' 'D' ' ' '#' '#' ' ' ' ' ' ' ' ' '#']
 ['#' ' ' ' ' '#' '#' ' ' ' ' '#' '#' '#']
 ['#' ' ' '#' ' ' '#' '#' ' ' ' ' '#' '#']
 ['#' ' ' ' ' ' ' ' ' ' ' '#' ' ' ' ' '#']
 ['#' ' ' ' ' ' ' '#' ' ' ' ' ' ' '#' '#']
 ['#' ' ' ' ' '#' ' ' ' ' ' ' '#' ' ' '#']
 ['#' ' ' ' ' ' ' ' ' '#' '#' '#' ' ' '#']
 ['#' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 'T' '#']
 ['#' '#' '#' '#' '#' '#' '#' '#' '#' '#']]
States visitable by the DFS Algorithm
[['#' '#' '#' '#' '#' '#' '#' '#' '#' '#']
 ['#' 'D' 'v' '#' '#' 'v' 'v' 'v' 'v' '#']
 ['#' 'v' 'v' '#' '#' 'v' 'v' '#' '#' '#']
 ['#' 'v' '#' 'v' '#' '#' 'v' 'v' '#' '#']
 ['#' 'v' 'v' 'v' 'v' 'v' '#' 'v' 'v' '#']
 ['#' 'v' 'v' 'v' '#' 'v' 'v' 'v' '#' '#']
 ['#' 'v' 'v' '#' 'v' 'v' 'v' '#' 'v' '#']
 ['#' 'v' 'v' 'v' 'v' '#' '#' '#' 'v' '#']
 ['#' 'v' 'v' 'v' 'v' 'v' 'v' 'v' 'T' '#']
 ['#' '#' '#' '#' '#' '#' '#' '#' '#' '#']]
The prospective layout is solvable


## **Return ONLY Solvable Layouts**

In [6]:
"""

Arguments: size of the layout, locations of the drone and the target, obstacle density, and the maximum number of iterations
           [def: 100 iterations]
Functionality: This function will consider, at most, the max_iter number of randomly generated layouts. If any prospective
               layout is solvable (checked using DFS), we return it after indicating the number of iterations it
               took to create that layout. If the obstacle density is high, hitting a candidate might not be possible.
Return: solvable layout or False (if no prospective layout was found)

"""


def strictly_solvable(layout_size, drone_location, target_location, obstacle_density = 0.2, max_iter = 100, verbose = True):
    
    for x in range(max_iter):
        prospective = generate_layout(layout_size, drone_location, target_location, obstacle_density) # Generate a 
                                                                                                      # prospective layout

        ######################################################################################################################

        raster_dict = raster_scan(prospective) # Obtain the dictionary with the raster indexation

        ######################################################################################################################

        AList = generate_adjacency_list(prospective, raster_dict) # Generate the adjacency list

        (visited, parent) = ({}, {}) # Set visit and parent as empty dictionaries
        visited, parent = DFSInitList(AList, visited, parent) # Populate these dictionaries given the adjacency list
        DFSList(AList, raster_dict[drone_location], visited, parent) # Implement DFS recursively

        ######################################################################################################################

        if visited[raster_dict[target_location]]: # If the layout is solvable
            if verbose:
                print("Iterations take: ", x)
                print(prospective)
            return prospective
    
    print("The obstacle density is very high! Consider reducing it")
    return False

###############################################################################################################################

# Test the code here

drone_location = (1, 1) # Example drone location
target_location = (5, 5) # Example target location
layout_size = (7, 7) # Example layout size
layout = strictly_solvable(layout_size, drone_location, target_location, obstacle_density = 0.5, verbose = True) # Generate a solvable layout

Iterations take:  6
[['#' '#' '#' '#' '#' '#' '#']
 ['#' 'D' '#' '#' ' ' '#' '#']
 ['#' ' ' '#' ' ' ' ' '#' '#']
 ['#' ' ' '#' ' ' ' ' ' ' '#']
 ['#' ' ' '#' '#' '#' ' ' '#']
 ['#' ' ' ' ' ' ' ' ' 'T' '#']
 ['#' '#' '#' '#' '#' '#' '#']]


# **Generating All the Test Cases**

There are 20 simulation scenarios that are considered. 
- The layout sizes are varied from 5 x 5 to 8 x 8 in steps of 1.

- The obstacle densities are varied from 0 to 0.4 in steps of 0.1.

Each simulation scenario admits 50 test cases + 1 test case for learning.

In [7]:
density = [0, 0.1, 0.2, 0.3, 0.4] # Density of obstacles
size = [5, 6, 7, 8] # Size of the layout
n_test_cases = 51 # Number of test cases
drone_location = (1, 1) # Example drone location
test_cases = [] # Store the test cases

for s in size:
    target_location = (s - 2, s - 2) # Example target location
    layout_size = (s, s) # Example layout size
    for d in density:
        temp = []
        for k in range(n_test_cases):
            layout = strictly_solvable(layout_size, drone_location, target_location, obstacle_density = d, verbose = False) # Generate a solvable layout
            temp.append(layout) # Append the layout to the list of test cases
        test_cases.append(temp) # Append the list of test cases to the main list

# <center>  **RL Simulations** </center>
### All the simulations were done using the [Gym framework](https://www.gymlibrary.dev/content/environment_creation)

### We use the following environment naming convention:

---

### MDP Characterization:

1. **Discrete:** *Disc*
2. **Continuous:** *Cont*

---

### Reward Characterization:

1. **Deterministic:** *Det*
2. **Probabilistic:** *Prob*

---

### Example:

For a **Discrete MDP** with a **Deterministic Reward Mechanism**, we use the naming: **DiscDetEnv**

## **Case 1: Discrete Markov Decision Process with Deterministic Reward**

### **Locally Optimal Action**

#### **Environment Definition**

In [8]:
class DiscDetEnv(gym.Env):
    
    def __init__(self, layout, drone_location, target_location, completion_reward = 100000, obstacle_hit_reward = -100000,
                         reduce_dist_reward = 10, increase_dist_reward = -5):

        self.layout = layout.copy() # Layout is made an attribute of the object so that it can be used in the methods
        self.rows, self.columns = np.shape(layout) # Obtain the number of rows and columns
        self.raster_dict = raster_scan(layout) # Obtain the dictionary with the raster indexation
        self.start = drone_location # Starting position of the drone
        self.target_location = target_location # Target/ Delivery location
        self.agent_location = self.start

        #######################################################################################################################
        
        # Rewards

        self.completion_reward = completion_reward
        self.obstacle_hit_reward = obstacle_hit_reward
        self.reduce_dist_reward = reduce_dist_reward
        self.increase_dist_reward = increase_dist_reward
        self.reduce_dist_reward = reduce_dist_reward

        #######################################################################################################################

        # Observations are dictionaries with the agent's and the target's location.

        """

        The -2 helps to characterize the true number of states. We want to exclude the possibility of both the target and
        agent (drone) from being present on the peripheral walls.
        Number of valid states <= (rows - 2) * (columns - 2)
        We prevent the drone from running over obstacles with the reward mechanism.
        
        """
        
        self.observation_space = spaces.Dict(
            {
                "agent": spaces.MultiDiscrete([self.rows, self.columns])
            }
        )

        #######################################################################################################################

        self.action_space = spaces.Discrete(4) # We have 4 actions, corresponding to "right", "up", "left", "down"

        """
        
        The following dictionary maps abstract actions from `self.action_space` to 
        the direction we will walk in if that action is taken.
        I.e., 0 corresponds to "right," 1 to "up," etc.
        
        """
        
        self.action_to_direction = {
            0: np.array([1, 0]),
            1: np.array([0, 1]),
            2: np.array([-1, 0]),
            3: np.array([0, -1]),
        }

    ###########################################################################################################################
    
    def reset(self):
        self.agent_location = self.start
        return {'agent': self.agent_location}

    ###########################################################################################################################

    def reward_mechanism(self, action):
            
            agent_coordinates = np.array([self.agent_location[0], self.agent_location[1]])
            target_coordinates = np.array([self.target_location[0], self.target_location[1]])

            ##################################################################################################################
            
            if action == 0:
                prospective_coordinates = [self.agent_location[0] + 1, self.agent_location[1]]
                if self.layout[self.agent_location[0] + 1][self.agent_location[1]] == '#':
                    return self.obstacle_hit_reward
                
                if self.layout[self.agent_location[0] + 1][self.agent_location[1]] == 'T':
                    return self.completion_reward
                
                else:
                    if np.linalg.norm(agent_coordinates - target_coordinates, ord = 1) < np.linalg.norm(prospective_coordinates - target_coordinates, ord = 1):
                        return self.increase_dist_reward
                    if np.linalg.norm(agent_coordinates - target_coordinates, ord = 1) > np.linalg.norm(prospective_coordinates - target_coordinates, ord = 1):
                        return self.reduce_dist_reward

            ##################################################################################################################

            if action == 1:
                prospective_coordinates = [self.agent_location[0], self.agent_location[1] + 1]
                if self.layout[self.agent_location[0]][self.agent_location[1] + 1] == '#':
                    return self.obstacle_hit_reward
                
                if self.layout[self.agent_location[0]][self.agent_location[1] + 1] == 'T':
                    return self.completion_reward

                else:
                    if np.linalg.norm(agent_coordinates - target_coordinates, ord = 1) < np.linalg.norm(prospective_coordinates - target_coordinates, ord = 1):
                        return self.increase_dist_reward
                    if np.linalg.norm(agent_coordinates - target_coordinates, ord = 1) > np.linalg.norm(prospective_coordinates - target_coordinates, ord = 1):
                        return self.reduce_dist_reward

            ##################################################################################################################

            if action == 2:
                prospective_coordinates = [self.agent_location[0] - 1, self.agent_location[1]]
                if self.layout[self.agent_location[0] - 1][self.agent_location[1]] == '#':
                    return self.obstacle_hit_reward
                
                if self.layout[self.agent_location[0] - 1][self.agent_location[1]] == 'T':
                    return self.completion_reward

                else:
                    if np.linalg.norm(agent_coordinates - target_coordinates, ord = 1) < np.linalg.norm(prospective_coordinates - target_coordinates, ord = 1):
                        return self.increase_dist_reward
                    if np.linalg.norm(agent_coordinates - target_coordinates, ord = 1) > np.linalg.norm(prospective_coordinates - target_coordinates, ord = 1):
                        return self.reduce_dist_reward

            ##################################################################################################################

            if action == 3:
                prospective_coordinates = [self.agent_location[0], self.agent_location[1] - 1]
                if self.layout[self.agent_location[0]][self.agent_location[1] - 1] == '#':
                    return self.obstacle_hit_reward
                
                if self.layout[self.agent_location[0]][self.agent_location[1] - 1] == 'T':
                    return self.completion_reward

                else:
                    if np.linalg.norm(agent_coordinates - target_coordinates, ord = 1) < np.linalg.norm(prospective_coordinates - target_coordinates, ord = 1):
                        return self.increase_dist_reward
                    if np.linalg.norm(agent_coordinates - target_coordinates, ord = 1) > np.linalg.norm(prospective_coordinates - target_coordinates, ord = 1):
                        return self.reduce_dist_reward

    ###########################################################################################################################
            
    def step(self):
        rewards_dict = {
                            0: self.reward_mechanism(0),
                            1: self.reward_mechanism(1),
                            2: self.reward_mechanism(2),
                            3: self.reward_mechanism(3)
                        }
        
        # Check if all the values are distinct. If they are, we can directly obtain the action with the maximum reward

        if len(rewards_dict.values()) == len(set(rewards_dict.values())):

            action = max(rewards_dict, key = rewards_dict.get)

        # If the values are not distinct, we need to break the tie. We will randomly select one of the actions with the maximum reward
        
        else:
            action = np.random.choice([key for key in rewards_dict.keys() if rewards_dict[key] == max(rewards_dict.values())])

        reward = rewards_dict[action]
        direction = self.action_to_direction[action] # Map the action to the walking direction
        self.layout[self.agent_location[0], self.agent_location[1]] = ' ' # The agent has left the current location

        self.agent_location = (self.agent_location[0] + direction[0], self.agent_location[1] + direction[1]) # Update the agent's location
        self.layout[self.agent_location[0], self.agent_location[1]] = 'D' # The agent has reached the new location
        
        # An episode is done iff the agent has reached the target
        if self.agent_location == self.target_location:
            terminated = True
        else:
            terminated = False


        return reward, terminated, action

    ###########################################################################################################################

    def render(self):
        present_layout = self.layout.copy()
        present_layout[self.start[0], self.start[1]] = ' '
        present_layout[self.agent_location[0], self.agent_location[1]] = 'D'
        print(present_layout)
        return

#### **Test Case: Where the Policy Works**

In [66]:
print("Initial Setup")
layout_size = (10, 10)
drone_location = (1, 1)
target_location = (8, 8)
layout = strictly_solvable(layout_size, drone_location, target_location, obstacle_density = 0.2) # Generate a solvable layout
layout_RL_Learning = layout.copy()

env = DiscDetEnv(layout, (1, 1), (8, 8))
state = env.reset()
terminated = False
counter = 0

while not terminated:
    reward, terminated, action = env.step()
    state = env.agent_location
    print(f"Action: {action}, New State: {state}, Reward: {reward}, Terminated: {terminated}")
    env.render()
    counter = counter + 1
    
print("Number of steps taken: ", counter)

Initial Setup
Iterations take:  0
[['#' '#' '#' '#' '#' '#' '#' '#' '#' '#']
 ['#' 'D' ' ' ' ' ' ' ' ' ' ' ' ' ' ' '#']
 ['#' ' ' ' ' ' ' '#' ' ' '#' ' ' '#' '#']
 ['#' '#' '#' ' ' ' ' ' ' ' ' '#' ' ' '#']
 ['#' ' ' ' ' '#' '#' ' ' ' ' ' ' ' ' '#']
 ['#' '#' ' ' ' ' ' ' ' ' ' ' ' ' ' ' '#']
 ['#' '#' ' ' '#' '#' ' ' ' ' ' ' ' ' '#']
 ['#' ' ' ' ' ' ' ' ' ' ' ' ' '#' ' ' '#']
 ['#' ' ' '#' ' ' ' ' ' ' '#' ' ' 'T' '#']
 ['#' '#' '#' '#' '#' '#' '#' '#' '#' '#']]
Action: 1, New State: (1, 2), Reward: 10, Terminated: False
[['#' '#' '#' '#' '#' '#' '#' '#' '#' '#']
 ['#' ' ' 'D' ' ' ' ' ' ' ' ' ' ' ' ' '#']
 ['#' ' ' ' ' ' ' '#' ' ' '#' ' ' '#' '#']
 ['#' '#' '#' ' ' ' ' ' ' ' ' '#' ' ' '#']
 ['#' ' ' ' ' '#' '#' ' ' ' ' ' ' ' ' '#']
 ['#' '#' ' ' ' ' ' ' ' ' ' ' ' ' ' ' '#']
 ['#' '#' ' ' '#' '#' ' ' ' ' ' ' ' ' '#']
 ['#' ' ' ' ' ' ' ' ' ' ' ' ' '#' ' ' '#']
 ['#' ' ' '#' ' ' ' ' ' ' '#' ' ' 'T' '#']
 ['#' '#' '#' '#' '#' '#' '#' '#' '#' '#']]
Action: 0, New State: (2, 2), Reward: 10, Te

#### **Test Case: Where the Policy Fails**

In [11]:
print("Initial Setup")
layout = strictly_solvable(layout_size, drone_location, target_location, obstacle_density = 0.5) # Generate a solvable layout
layout_RL_Learning = layout.copy()

env = DiscDetEnv(layout, (1, 1), (8, 8))
state = env.reset()
terminated = False
counter = 0

while not terminated:
    reward, terminated, action = env.step()
    state = env.agent_location
    print(f"Action: {action}, New State: {state}, Reward: {reward}, Terminated: {terminated}")
    env.render()
    counter = counter + 1

    if counter > 100:
        break
    
print("Number of steps taken: ", counter)

Initial Setup
Iterations take:  6
[['#' '#' '#' '#' '#' '#' '#' '#' '#' '#']
 ['#' 'D' ' ' '#' '#' '#' ' ' '#' '#' '#']
 ['#' ' ' ' ' '#' ' ' ' ' '#' '#' '#' '#']
 ['#' ' ' '#' '#' ' ' ' ' ' ' ' ' '#' '#']
 ['#' ' ' ' ' ' ' ' ' ' ' ' ' ' ' '#' '#']
 ['#' ' ' '#' '#' '#' ' ' '#' ' ' ' ' '#']
 ['#' ' ' ' ' '#' '#' '#' '#' '#' ' ' '#']
 ['#' '#' ' ' '#' ' ' ' ' '#' ' ' ' ' '#']
 ['#' ' ' ' ' '#' ' ' '#' '#' '#' 'T' '#']
 ['#' '#' '#' '#' '#' '#' '#' '#' '#' '#']]
Action: 0, New State: (2, 1), Reward: 1, Terminated: False
[['#' '#' '#' '#' '#' '#' '#' '#' '#' '#']
 ['#' ' ' ' ' '#' '#' '#' ' ' '#' '#' '#']
 ['#' 'D' ' ' '#' ' ' ' ' '#' '#' '#' '#']
 ['#' ' ' '#' '#' ' ' ' ' ' ' ' ' '#' '#']
 ['#' ' ' ' ' ' ' ' ' ' ' ' ' ' ' '#' '#']
 ['#' ' ' '#' '#' '#' ' ' '#' ' ' ' ' '#']
 ['#' ' ' ' ' '#' '#' '#' '#' '#' ' ' '#']
 ['#' '#' ' ' '#' ' ' ' ' '#' ' ' ' ' '#']
 ['#' ' ' ' ' '#' ' ' '#' '#' '#' 'T' '#']
 ['#' '#' '#' '#' '#' '#' '#' '#' '#' '#']]
Action: 0, New State: (3, 1), Reward: 1, Term

### **Policy using RL Algorithms**

We re-define the environment to make it compatible with the library Stable Baselines3 which offers a high-level interface to train and evaluate RL agents.

#### **Compatible Environment Definition**

In [9]:
class DiscDetCompatEnv(gym.Env):
    def __init__(self, layout, drone_location, target_location, completion_reward = 100000, obstacle_hit_reward = -100000,
                 reduce_dist_reward = 10, increase_dist_reward = -5):
        super(DiscDetCompatEnv, self).__init__()  # Ensure compatibility by calling the parent constructor

        self.layout = layout # Layout is made an attribute of the object so that it can be used in the methods
        self.layout_duplicate = layout.copy() # Duplicate layout to prevent overwriting
        self.rows, self.columns = np.shape(layout) # Obtain the number of rows and columns
        self.raster_dict = raster_scan(layout) # Obtain the dictionary with the raster indexation
        self.start = drone_location # Starting position of the drone
        self.target_location = target_location # Target/ Delivery location
        self.agent_location = self.start # Initialize the agent location
        # self.step_count = 0 # Initialize the step count (optional)

        #######################################################################################################################
        
        # Rewards

        self.completion_reward = completion_reward
        self.obstacle_hit_reward = obstacle_hit_reward
        self.reduce_dist_reward = reduce_dist_reward
        self.increase_dist_reward = increase_dist_reward
        self.reduce_dist_reward = reduce_dist_reward

        #######################################################################################################################

        # Observations are dictionaries with the agent's location. Note that the target's location is fixed.

        """
        
        We prevent the drone from running over obstacles with the reward mechanism.
        
        """
        
        self.observation_space = spaces.Dict(
            {
                "agent": spaces.MultiDiscrete([self.rows, self.columns])
            }
        )

        #######################################################################################################################

        self.action_space = spaces.Discrete(4) # We have 4 actions, corresponding to "right", "up", "left", "down"

        """
        
        The following dictionary maps abstract actions from `self.action_space` to 
        the direction we will walk in if that action is taken.
        I.e., 0 corresponds to "right," 1 to "up," etc.
        
        """
        
        self.action_to_direction = {
            0: np.array([1, 0]),
            1: np.array([0, 1]),
            2: np.array([-1, 0]),
            3: np.array([0, -1]),
        }

    ###########################################################################################################################    

    def reset(self, seed = None, **kwargs):
    
        super().reset(seed = seed) # Set the seed for reproducibility
        if seed is not None:
            np.random.seed(seed)

        self.agent_location = self.start # Reset agent location
        self.layout = self.layout_duplicate.copy() # Reset the layout

        # Return the observation and an empty dictionary as additional info (as required by Gym)
        return {'agent': np.array(self.agent_location)}, {}
    
    ###########################################################################################################################

    def step(self, action):
        action = int(action)  # Ensure action is an integer
        reward = self.reward_mechanism(action)
        if reward == self.obstacle_hit_reward:
            terminated = False
            truncated = False
            observation = {'agent': np.array(self.agent_location)}
            info = {}  # Add any relevant information here (e.g., additional metrics)
            return observation, reward, terminated, truncated, info
        
        direction = self.action_to_direction[action]  # Map the action to the walking direction

        # layout_copy = self.layout.copy()
        self.layout[self.agent_location[0], self.agent_location[1]] = ' '  # Clear current location
        self.agent_location =   (
                                    np.clip(self.agent_location[0] + direction[0], 1, self.rows - 2), 
                                    np.clip(self.agent_location[1] + direction[1], 1, self.columns - 2)
                                )  # Update the agent location
        self.layout[self.agent_location[0], self.agent_location[1]] = 'D'  # Mark the new location

        # An episode is done if the agent has reached the target
        if self.agent_location == self.target_location:
            terminated = True
        else:
            terminated = False
        
        truncated = False # We are not truncating episodes based on the step count

        # Return observation, reward, terminated, truncated, and info
        observation = {'agent': np.array(self.agent_location)}
        info = {}  # Add any relevant information here (e.g., additional metrics)
        
        return observation, reward, terminated, truncated, info

    ###########################################################################################################################

    def reward_mechanism(self, action):
        agent_coordinates = np.array([self.agent_location[0], self.agent_location[1]])
        target_coordinates = np.array([self.target_location[0], self.target_location[1]])

        if action == 0:
            prospective_coordinates = [self.agent_location[0] + 1, self.agent_location[1]]
            if prospective_coordinates[0] >= self.rows:
                prospective_coordinates[0] = self.rows - 1

            ##################################################################################################################
            
            if self.layout[prospective_coordinates[0]][self.agent_location[1]] == '#':
                return self.obstacle_hit_reward
            if self.layout[prospective_coordinates[0]][self.agent_location[1]] == 'T':
                return self.completion_reward
            return self._calculate_distance_reward(agent_coordinates, prospective_coordinates, target_coordinates)

        # Repeat similar logic for other actions (1, 2, 3)
        
        if action == 1:
            prospective_coordinates = [self.agent_location[0], self.agent_location[1] + 1]
            if prospective_coordinates[1] >= self.columns:
                prospective_coordinates[1] = self.columns - 1
            if self.layout[self.agent_location[0]][prospective_coordinates[1]] == '#':
                return self.obstacle_hit_reward
            if self.layout[self.agent_location[0]][prospective_coordinates[1]] == 'T':
                return self.completion_reward
            return self._calculate_distance_reward(agent_coordinates, prospective_coordinates, target_coordinates)

        if action == 2:
            prospective_coordinates = [self.agent_location[0] - 1, self.agent_location[1]]
            if prospective_coordinates[0] < 0:
                prospective_coordinates[0] = 0
            if self.layout[prospective_coordinates[0]][self.agent_location[1]] == '#':
                return self.obstacle_hit_reward
            if self.layout[prospective_coordinates[0]][self.agent_location[1]] == 'T':
                return self.completion_reward
            return self._calculate_distance_reward(agent_coordinates, prospective_coordinates, target_coordinates)

        if action == 3:
            prospective_coordinates = [self.agent_location[0], self.agent_location[1] - 1]
            if prospective_coordinates[1] < 0:
                prospective_coordinates[1] = 0
            if self.layout[self.agent_location[0]][prospective_coordinates[1]] == '#':
                return self.obstacle_hit_reward
            if self.layout[self.agent_location[0]][prospective_coordinates[1]] == 'T':
                return self.completion_reward
            return self._calculate_distance_reward(agent_coordinates, prospective_coordinates, target_coordinates)

    def _calculate_distance_reward(self, agent_coordinates, prospective_coordinates, target_coordinates):
        if np.linalg.norm(agent_coordinates - target_coordinates, ord=1) < np.linalg.norm(prospective_coordinates - target_coordinates, ord=1):
            return self.increase_dist_reward
        return self.reduce_dist_reward

    ###########################################################################################################################

    def render(self, mode = 'human'):
        print(self.layout)  # Print the layout with the agent's current position
        return

#### **Training Phase**

In [35]:
print("Initial Setup")
env = DiscDetCompatEnv(layout_RL_Learning, drone_location, target_location) # Load the environment
print(env.layout)

# Instantiate the model with improved hyperparameters
model = DQN('MultiInputPolicy', env, 
            verbose=1,
            learning_rate = 1e-2,    # Slightly lower learning rate
            buffer_size = 100000,    # Increase buffer size for more experience replay
            batch_size = 32,         # Batch size for more stable training
            learning_starts = 100,  # Start learning after 100 timesteps
            exploration_fraction = 0.5,  # Keep exploration high initially
            exploration_final_eps = 0.25,  # Final epsilon after decay
            gamma = 0.25)  # Discount factor (gamma) for the future reward

model.learn(total_timesteps = 20000, log_interval = 4) # Train the model with more timesteps and log interval for progress

Initial Setup
[['#' '#' '#' '#' '#' '#' '#' '#' '#' '#']
 ['#' 'D' ' ' ' ' ' ' '#' '#' ' ' ' ' '#']
 ['#' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' '#']
 ['#' '#' ' ' '#' ' ' ' ' ' ' ' ' ' ' '#']
 ['#' '#' ' ' '#' ' ' ' ' ' ' '#' ' ' '#']
 ['#' '#' ' ' ' ' ' ' ' ' ' ' ' ' ' ' '#']
 ['#' ' ' ' ' '#' '#' '#' ' ' ' ' ' ' '#']
 ['#' ' ' ' ' ' ' '#' '#' ' ' ' ' ' ' '#']
 ['#' ' ' '#' '#' '#' ' ' ' ' ' ' 'T' '#']
 ['#' '#' '#' '#' '#' '#' '#' '#' '#' '#']]
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
-----------------------------------
| rollout/            |           |
|    ep_len_mean      | 216       |
|    ep_rew_mean      | -5.35e+06 |
|    exploration_rate | 0.935     |
| time/               |           |
|    episodes         | 4         |
|    fps              | 547       |
|    time_elapsed     | 1         |
|    total_timesteps  | 862       |
| train/              |           |
|    learning_rate    | 0.01      |
|    loss             | 2.17e

#### **Testing Phase**

In [39]:
print("Initial Setup")
print(layout_RL_Learning)

# Test the model here

obs = env.reset()
print("Initial State: ", obs[0]['agent'])
counter = 0
for _ in range(1000):
    action, _states = model.predict(obs[0])
    observation, reward, terminated, truncated, info = env.step(action)
    print(f"Action: {action}, New State: {observation['agent']}, Reward: {reward}, Terminated: {terminated}")
    env.render()
    counter = counter + 1
    if terminated:
        break

print("Number of steps taken: ", counter)

Initial Setup
[['#' '#' '#' '#' '#' '#' '#' '#' '#' '#']
 ['#' 'D' ' ' ' ' ' ' '#' '#' ' ' ' ' '#']
 ['#' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' '#']
 ['#' '#' ' ' '#' ' ' ' ' ' ' ' ' ' ' '#']
 ['#' '#' ' ' '#' ' ' ' ' ' ' '#' ' ' '#']
 ['#' '#' ' ' ' ' ' ' ' ' ' ' ' ' ' ' '#']
 ['#' ' ' ' ' '#' '#' '#' ' ' ' ' ' ' '#']
 ['#' ' ' ' ' ' ' '#' '#' ' ' ' ' ' ' '#']
 ['#' ' ' '#' '#' '#' ' ' ' ' ' ' 'T' '#']
 ['#' '#' '#' '#' '#' '#' '#' '#' '#' '#']]
Initial State:  [1 1]
Action: 3, New State: [1 1], Reward: -100000, Terminated: False
[['#' '#' '#' '#' '#' '#' '#' '#' '#' '#']
 ['#' 'D' ' ' ' ' ' ' '#' '#' ' ' ' ' '#']
 ['#' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' '#']
 ['#' '#' ' ' '#' ' ' ' ' ' ' ' ' ' ' '#']
 ['#' '#' ' ' '#' ' ' ' ' ' ' '#' ' ' '#']
 ['#' '#' ' ' ' ' ' ' ' ' ' ' ' ' ' ' '#']
 ['#' ' ' ' ' '#' '#' '#' ' ' ' ' ' ' '#']
 ['#' ' ' ' ' ' ' '#' '#' ' ' ' ' ' ' '#']
 ['#' ' ' '#' '#' '#' ' ' ' ' ' ' 'T' '#']
 ['#' '#' '#' '#' '#' '#' '#' '#' '#' '#']]
Action: 0, New State: [2 1], Reward: 1

## **Case 2: Discrete Markov Decision Process with Probabilistic Reward**

### **Environment Definition**

In [10]:
class DiscProbEnv(gym.Env):
    def __init__(self, layout, drone_location, target_location, completion_reward = 100000, obstacle_hit_reward = -100000,
                 reduce_dist_reward = 10, increase_dist_reward = -5, stay_reward = -1, wind_speed_bound_x = (-1, 1), wind_speed_bound_y = (-1, 1)):
        super(DiscProbEnv, self).__init__()  # Ensure compatibility by calling the parent constructor

        self.layout = layout # Layout is made an attribute of the object so that it can be used in the methods
        self.layout_duplicate = layout.copy() # Duplicate layout to prevent overwriting
        self.rows, self.columns = np.shape(layout) # Obtain the number of rows and columns
        self.raster_dict = raster_scan(layout) # Obtain the dictionary with the raster indexation
        self.start = drone_location # Starting position of the drone
        self.target_location = target_location # Target/ Delivery location
        self.agent_location = self.start # Initialize the agent location
        self.wind_speed_bound_x = wind_speed_bound_x # Wind speed bound in the x-direction
        self.wind_speed_bound_y = wind_speed_bound_y # Wind speed bound in the y-direction
        # self.step_count = 0 # Initialize the step count (optional)

        #######################################################################################################################
        
        # Rewards

        self.completion_reward = completion_reward
        self.obstacle_hit_reward = obstacle_hit_reward
        self.reduce_dist_reward = reduce_dist_reward
        self.increase_dist_reward = increase_dist_reward
        self.reduce_dist_reward = reduce_dist_reward
        self.stay_reward = stay_reward

        #######################################################################################################################

        # Observations are dictionaries with the agent's location. Note that the target's location is fixed.

        """
        
        We prevent the drone from running over obstacles with the reward mechanism.
        
        """
        
        self.observation_space = spaces.Dict(
            {
                "agent": spaces.MultiDiscrete([self.rows, self.columns])
            }
        )

        #######################################################################################################################

        self.action_space = spaces.Discrete(5) # We have 5 actions, corresponding to "right", "up", "left", "down", and "stay put"

        """
        
        The following dictionary maps abstract actions from `self.action_space` to 
        the direction we will walk in if that action is taken.
        I.e., 0 corresponds to "right," 1 to "up" etc.
        
        """
        
        self.action_to_direction = {
            0: np.array([1, 0]),
            1: np.array([0, 1]),
            2: np.array([-1, 0]),
            3: np.array([0, -1]),
            4: np.array([0, 0])
        }

    ###########################################################################################################################    

    def reset(self, seed = None, **kwargs):
    
        super().reset(seed = seed) # Set the seed for reproducibility
        if seed is not None:
            np.random.seed(seed)

        self.agent_location = self.start # Reset agent location
        self.layout = self.layout_duplicate.copy() # Reset the layout

        # Return the observation and an empty dictionary as additional info (as required by Gym)
        return {'agent': np.array(self.agent_location)}, {}
    
    ###########################################################################################################################

    def step(self, action):
        action = int(action)  # Ensure action is an integer
        reward = self.reward_mechanism(action)
        if reward == self.obstacle_hit_reward:
            terminated = False
            truncated = False
            observation = {'agent': np.array(self.agent_location)}
            info = {}  # Add any relevant information here (e.g., additional metrics)
            return observation, reward, terminated, truncated, info
        
        direction = self.action_to_direction[action]  # Map the action to the walking direction

        # layout_copy = self.layout.copy()
        self.layout[self.agent_location[0], self.agent_location[1]] = ' '  # Clear current location
        self.agent_location =   (
                                    np.clip(self.agent_location[0] + direction[0], 1, self.rows - 2), 
                                    np.clip(self.agent_location[1] + direction[1], 1, self.columns - 2)
                                )  # Update the agent location
        self.layout[self.agent_location[0], self.agent_location[1]] = 'D'  # Mark the new location

        # An episode is done if the agent has reached the target
        if self.agent_location == self.target_location:
            terminated = True
        else:
            terminated = False
        
        truncated = False # We are not truncating episodes based on the step count

        # Return observation, reward, terminated, truncated, and info
        observation = {'agent': np.array(self.agent_location)}
        info = {}  # Add any relevant information here (e.g., additional metrics)
        
        return observation, reward, terminated, truncated, info

    ###########################################################################################################################

    def reward_mechanism(self, action):
        agent_coordinates = np.array([self.agent_location[0], self.agent_location[1]])
        target_coordinates = np.array([self.target_location[0], self.target_location[1]])

        if action == 0:
            prospective_coordinates = [self.agent_location[0] + 1, self.agent_location[1]]
            if prospective_coordinates[0] >= self.rows:
                prospective_coordinates[0] = self.rows - 1

            ##################################################################################################################
            
            if self.layout[prospective_coordinates[0]][self.agent_location[1]] == '#':
                return self.obstacle_hit_reward
            if self.layout[prospective_coordinates[0]][self.agent_location[1]] == 'T':
                return self.completion_reward
            return self._calculate_distance_reward(agent_coordinates, prospective_coordinates, target_coordinates) + np.random.uniform(self.wind_speed_bound_x[0], self.wind_speed_bound_x[1])

        # Repeat similar logic for other actions (1, 2, 3)
        
        if action == 1:
            prospective_coordinates = [self.agent_location[0], self.agent_location[1] + 1]
            if prospective_coordinates[1] >= self.columns:
                prospective_coordinates[1] = self.columns - 1
            if self.layout[self.agent_location[0]][prospective_coordinates[1]] == '#':
                return self.obstacle_hit_reward
            if self.layout[self.agent_location[0]][prospective_coordinates[1]] == 'T':
                return self.completion_reward
            return self._calculate_distance_reward(agent_coordinates, prospective_coordinates, target_coordinates) + np.random.uniform(self.wind_speed_bound_y[0], self.wind_speed_bound_y[1])

        if action == 2:
            prospective_coordinates = [self.agent_location[0] - 1, self.agent_location[1]]
            if prospective_coordinates[0] < 0:
                prospective_coordinates[0] = 0
            if self.layout[prospective_coordinates[0]][self.agent_location[1]] == '#':
                return self.obstacle_hit_reward
            if self.layout[prospective_coordinates[0]][self.agent_location[1]] == 'T':
                return self.completion_reward
            return self._calculate_distance_reward(agent_coordinates, prospective_coordinates, target_coordinates) + np.random.uniform(self.wind_speed_bound_x[0], self.wind_speed_bound_x[1])

        if action == 3:
            prospective_coordinates = [self.agent_location[0], self.agent_location[1] - 1]
            if prospective_coordinates[1] < 0:
                prospective_coordinates[1] = 0
            if self.layout[self.agent_location[0]][prospective_coordinates[1]] == '#':
                return self.obstacle_hit_reward
            if self.layout[self.agent_location[0]][prospective_coordinates[1]] == 'T':
                return self.completion_reward
            return self._calculate_distance_reward(agent_coordinates, prospective_coordinates, target_coordinates) + np.random.uniform(self.wind_speed_bound_y[0], self.wind_speed_bound_y[1])
        
        if action == 4:
            return self.stay_reward

    def _calculate_distance_reward(self, agent_coordinates, prospective_coordinates, target_coordinates):
        if np.linalg.norm(agent_coordinates - target_coordinates, ord=1) < np.linalg.norm(prospective_coordinates - target_coordinates, ord = 1):
            return self.increase_dist_reward
        return self.reduce_dist_reward

    ###########################################################################################################################

    def render(self, mode = 'human'):
        print(self.layout)  # Print the layout with the agent's current position
        return

### **Training Phase**

In [95]:
print("Initial Setup")
layout_size = (8, 8)
drone_location = (1, 1)
target_location = (6, 6)
layout_RL_Learning = strictly_solvable(layout_size, drone_location, target_location, obstacle_density = 0.1) # Generate a solvable layout
env = DiscProbEnv(layout_RL_Learning, drone_location, target_location) # Load the environment
print(env.layout)

# Instantiate the model with improved hyperparameters
# model = DQN('MultiInputPolicy', env, 
#             verbose=1,
#             learning_rate = 1e-2,    # Slightly lower learning rate
#             buffer_size = 100000,    # Increase buffer size for more experience replay
#             batch_size = 32,         # Batch size for more stable training
#             learning_starts = 100,  # Start learning after 100 timesteps
#             exploration_fraction = 0.5,  # Keep exploration high initially
#             exploration_final_eps = 0.25,  # Final epsilon after decay
#             gamma = 0.8)  # Discount factor (gamma) for the future reward

model = DQN('MultiInputPolicy', env, verbose=1)

model.learn(total_timesteps = 10000, log_interval = 4) # Train the model with more timesteps and log interval for progress

Initial Setup
Iterations take:  0
[['#' '#' '#' '#' '#' '#' '#' '#']
 ['#' 'D' ' ' '#' ' ' ' ' ' ' '#']
 ['#' ' ' ' ' ' ' ' ' ' ' ' ' '#']
 ['#' ' ' ' ' ' ' '#' ' ' ' ' '#']
 ['#' ' ' ' ' ' ' ' ' ' ' ' ' '#']
 ['#' '#' '#' ' ' ' ' ' ' ' ' '#']
 ['#' ' ' ' ' ' ' ' ' ' ' 'T' '#']
 ['#' '#' '#' '#' '#' '#' '#' '#']]
[['#' '#' '#' '#' '#' '#' '#' '#']
 ['#' 'D' ' ' '#' ' ' ' ' ' ' '#']
 ['#' ' ' ' ' ' ' ' ' ' ' ' ' '#']
 ['#' ' ' ' ' ' ' '#' ' ' ' ' '#']
 ['#' ' ' ' ' ' ' ' ' ' ' ' ' '#']
 ['#' '#' '#' ' ' ' ' ' ' ' ' '#']
 ['#' ' ' ' ' ' ' ' ' ' ' 'T' '#']
 ['#' '#' '#' '#' '#' '#' '#' '#']]
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 105      |
|    ep_rew_mean      | -2.3e+06 |
|    exploration_rate | 0.601    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 933      |
|    time_elapsed     | 0     

### **Testing Phase**

In [97]:
print("Initial Setup")
print(layout_RL_Learning)

# Test the model here

obs = env.reset()
print("Initial State: ", obs[0]['agent'])
counter = 0
for _ in range(100):
    action, _states = model.predict(obs[0])
    observation, reward, terminated, truncated, info = env.step(action)
    print(f"Action: {action}, New State: {observation['agent']}, Reward: {reward}, Terminated: {terminated}")
    env.render()
    counter = counter + 1
    if terminated:
        break

print("Number of steps taken: ", counter)

Initial Setup
[['#' '#' '#' '#' '#' '#' '#' '#']
 ['#' 'D' ' ' '#' ' ' ' ' ' ' '#']
 ['#' ' ' ' ' ' ' ' ' ' ' ' ' '#']
 ['#' ' ' ' ' ' ' '#' ' ' ' ' '#']
 ['#' ' ' ' ' ' ' ' ' ' ' ' ' '#']
 ['#' '#' '#' ' ' ' ' ' ' ' ' '#']
 ['#' ' ' ' ' ' ' ' ' ' ' 'T' '#']
 ['#' '#' '#' '#' '#' '#' '#' '#']]
Initial State:  [1 1]
Action: 4, New State: [1 1], Reward: -1, Terminated: False
[['#' '#' '#' '#' '#' '#' '#' '#']
 ['#' 'D' ' ' '#' ' ' ' ' ' ' '#']
 ['#' ' ' ' ' ' ' ' ' ' ' ' ' '#']
 ['#' ' ' ' ' ' ' '#' ' ' ' ' '#']
 ['#' ' ' ' ' ' ' ' ' ' ' ' ' '#']
 ['#' '#' '#' ' ' ' ' ' ' ' ' '#']
 ['#' ' ' ' ' ' ' ' ' ' ' 'T' '#']
 ['#' '#' '#' '#' '#' '#' '#' '#']]
Action: 4, New State: [1 1], Reward: -1, Terminated: False
[['#' '#' '#' '#' '#' '#' '#' '#']
 ['#' 'D' ' ' '#' ' ' ' ' ' ' '#']
 ['#' ' ' ' ' ' ' ' ' ' ' ' ' '#']
 ['#' ' ' ' ' ' ' '#' ' ' ' ' '#']
 ['#' ' ' ' ' ' ' ' ' ' ' ' ' '#']
 ['#' '#' '#' ' ' ' ' ' ' ' ' '#']
 ['#' ' ' ' ' ' ' ' ' ' ' 'T' '#']
 ['#' '#' '#' '#' '#' '#' '#' '#']]
Act

## **Case 3: Continuous Markov Decision Process with Deterministic Reward**

### **Environment Definition**

In [11]:
class ContDetEnv(gym.Env):
    def __init__(self, layout, drone_location, target_location, initial_velocity = np.array([0, 0]), completion_reward = 100000, 
                 obstacle_hit_reward = -100000, reduce_dist_reward = 10, increase_dist_reward = -5, stand_still_reward = -1, speed_limit_x = 5,
                 speed_limit_y = 5, acceleration_limit_x = 2, acceleration_limit_y = 2, time_step = 0.5, distance_threshold = 0.1):
        super(ContDetEnv, self).__init__()  # Ensure compatibility by calling the parent constructor

        self.layout = layout # Layout is made an attribute of the object so that it can be used in the methods
        self.layout_duplicate = layout.copy() # Duplicate layout to prevent overwriting
        self.rows, self.columns = np.shape(layout) # Obtain the number of rows and columns
        self.raster_dict = raster_scan(layout) # Obtain the dictionary with the raster indexation
        self.start = drone_location # Starting position of the drone
        self.target_location = np.array(target_location) # Target/ Delivery location
        self.agent_location = np.array([self.start[0], self.start[1]]) # Initialize the agent location
        # self.step_count = 0 # Initialize the step count (optional)
        self.velocity = initial_velocity # Initialize the velocity of the agent
        self.speed_limit_x = speed_limit_x # Speed limit in the x-direction
        self.speed_limit_y = speed_limit_y # Speed limit in the y-direction
        self.time_step = time_step # Time step for the simulation
        self.distance_threshold = distance_threshold # Distance threshold for the agent to reach the target

        #######################################################################################################################
        
        # Rewards

        self.completion_reward = completion_reward
        self.obstacle_hit_reward = obstacle_hit_reward
        self.reduce_dist_reward = reduce_dist_reward
        self.increase_dist_reward = increase_dist_reward
        self.reduce_dist_reward = reduce_dist_reward
        self.stand_still_reward = stand_still_reward

        #######################################################################################################################

        # Observations are dictionaries with the agent's location. Note that the target's location is fixed.

        """
        
        We prevent the drone from running over obstacles with the reward mechanism.
        
        """

        self.observation_space = spaces.Dict({
            "agent_position": spaces.Box(low=np.array([0, 0]), high=np.array([self.rows, self.columns]), dtype = np.float32),
            "agent_velocity": spaces.Box(low = np.array([-self.speed_limit_x, -self.speed_limit_y]), 
                                   high = np.array([self.speed_limit_x, self.speed_limit_y]), dtype = np.float32)
        })

        #######################################################################################################################

        self.action_space = spaces.Box(low = np.array([-acceleration_limit_x, -acceleration_limit_y]),
                                        high = np.array([acceleration_limit_x, acceleration_limit_y]), dtype = np.float32)

    ###########################################################################################################################    

    def reset(self, seed = None, **kwargs):
    
        super().reset(seed = seed) # Set the seed for reproducibility
        if seed is not None:
            np.random.seed(seed)

        self.agent_location = self.start # Reset agent location
        self.velocity = np.array([0, 0]) # Reset the velocity
        self.layout = self.layout_duplicate.copy() # Reset the layout

        # Return the observation and an empty dictionary as additional info (as required by Gym)
        return {'agent_position': np.array(self.agent_location), 'agent_velocity': np.array(self.velocity)}, {}
    
    ###########################################################################################################################

    def step(self, action):

        position = self.agent_location
        velocity = self.velocity
        acceleration_x, acceleration_y = action
        
        # Update velocity and position
        velocity[0] = velocity[0] + (acceleration_x * self.time_step)
        velocity[1] = velocity[1] + (acceleration_y * self.time_step)
        position_new = position + (velocity * self.time_step)

        # Check if there are obstacles within the distance_threshold from the new position
        
        possible_obstacle_coordinates = {
                                            "right_up": (np.ceil(position_new[0]), np.ceil(position_new[1])),
                                            "right_down": (np.ceil(position_new[0]), np.floor(position_new[1])),
                                            "left_up": (np.floor(position_new[0]), np.ceil(position_new[1])),
                                            "left_down": (np.floor(position_new[0]), np.floor(position_new[1]))
                                        }

        for value in possible_obstacle_coordinates.values():
            if value[0] >= self.rows or value[1] >= self.columns or value[0] < 0 or value[1] < 0:
                reward = self.obstacle_hit_reward
                terminated = False
                truncated = False
                observation = {'agent_position': position, 'agent_velocity': velocity}
                info = {}  # Add any relevant information here (e.g., additional metrics)
                return observation, reward, terminated, truncated, info
            if self.layout[int(value[0]), int(value[1])] == '#':
                coordinate = np.array([value[0], value[1]])
                if np.linalg.norm(position_new - coordinate) < self.distance_threshold:
                    reward = self.obstacle_hit_reward
                    terminated = False
                    truncated = False
                    observation = {'agent_position': position, 'agent_velocity': velocity}
                    info = {}  # Add any relevant information here (e.g., additional metrics)
                    return observation, reward, terminated, truncated, info

        # Update the agent's position and velocity

        self.agent_location = position_new
        self.velocity = velocity
        if np.linalg.norm(self.agent_location - self.target_location) < np.linalg.norm(position - self.target_location):
            reward = self.reduce_dist_reward
            terminated = False
            truncated = False
            observation = {'agent_position': position_new, 'agent_velocity': velocity}
            info = {}
            return observation, reward, terminated, truncated, info
        
        if np.linalg.norm(self.agent_location - self.target_location) > np.linalg.norm(position - self.target_location):
            reward = self.increase_dist_reward
            terminated = False
            truncated = False
            observation = {'agent_position': position_new, 'agent_velocity': velocity}
            info = {}
            return observation, reward, terminated, truncated, info

        # An episode is done if the agent is within the distance threshold from the target

        if np.linalg.norm(self.agent_location - self.target_location) < self.distance_threshold:
            terminated = True
            truncated = False
            reward = self.completion_reward
            observation = {'agent_position': np.array(self.agent_location), 'agent_velocity': self.velocity}
            info = {}  # Add any relevant information here (e.g., additional metrics)
            return observation, reward, terminated, truncated, info
        
        terminated = False
        truncated = False
        reward = self.stand_still_reward
        observation = {'agent_position': np.array(self.agent_location), 'agent_velocity': self.velocity}
        info = {}  # Add any relevant information here (e.g., additional metrics)
        return observation, reward, terminated, truncated, info

    ###########################################################################################################################

    def render(self, mode = 'human'):
        print(self.layout)  # Print the layout with the agent's current position
        print("Velocity of the Agent: ", self.velocity) # Print the velocity of the agent
        print("Position of the Agent: ", self.agent_location) # Print the agent's location
        return

### **Training Phase**

In [12]:
layout_size = (8, 8)
drone_location = (1, 1)
target_location = (6, 6)
layout_RL_Learning = strictly_solvable(layout_size, drone_location, target_location, obstacle_density = 0.2) # Generate a solvable layout
print("Initial Setup")
env = ContDetEnv(layout_RL_Learning, drone_location, target_location) # Load the environment
print(env.layout)

# Instantiate the model with improved hyperparameters
model = PPO('MultiInputPolicy', env, verbose = 1)

model.learn(total_timesteps = 10000, log_interval = 10) # Train the model with more timesteps and log interval for progress

Iterations take:  0
[['#' '#' '#' '#' '#' '#' '#' '#']
 ['#' 'D' ' ' '#' ' ' '#' ' ' '#']
 ['#' ' ' ' ' ' ' ' ' '#' ' ' '#']
 ['#' ' ' ' ' ' ' ' ' '#' ' ' '#']
 ['#' ' ' '#' ' ' '#' ' ' '#' '#']
 ['#' ' ' '#' ' ' ' ' ' ' ' ' '#']
 ['#' ' ' ' ' ' ' ' ' ' ' 'T' '#']
 ['#' '#' '#' '#' '#' '#' '#' '#']]
Initial Setup
[['#' '#' '#' '#' '#' '#' '#' '#']
 ['#' 'D' ' ' '#' ' ' '#' ' ' '#']
 ['#' ' ' ' ' ' ' ' ' '#' ' ' '#']
 ['#' ' ' ' ' ' ' ' ' '#' ' ' '#']
 ['#' ' ' '#' ' ' '#' ' ' '#' '#']
 ['#' ' ' '#' ' ' ' ' ' ' ' ' '#']
 ['#' ' ' ' ' ' ' ' ' ' ' 'T' '#']
 ['#' '#' '#' '#' '#' '#' '#' '#']]
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


### **Testing Phase**

In [13]:
print("Initial Setup")
print(layout_RL_Learning)

# Test the model here

obs = env.reset()
counter = 0
for _ in range(10000):
    action, _states = model.predict(obs[0])
    observation, reward, terminated, truncated, info = env.step(action)
    print(f"Action: {action}, New Position: {observation['agent_position']}, New Velocity: {observation['agent_velocity']}, Reward: {reward}, Terminated: {terminated}")
    # env.render()
    counter = counter + 1
    if terminated:
        break

print("Number of steps taken: ", counter)

Initial Setup
[['#' '#' '#' '#' '#' '#' '#' '#']
 ['#' 'D' ' ' '#' ' ' '#' ' ' '#']
 ['#' ' ' ' ' ' ' ' ' '#' ' ' '#']
 ['#' ' ' ' ' ' ' ' ' '#' ' ' '#']
 ['#' ' ' '#' ' ' '#' ' ' '#' '#']
 ['#' ' ' '#' ' ' ' ' ' ' ' ' '#']
 ['#' ' ' ' ' ' ' ' ' ' ' 'T' '#']
 ['#' '#' '#' '#' '#' '#' '#' '#']]
Action: [-0.14763476  0.2911046 ], New Position: [1. 1.], New Velocity: [0 0], Reward: -1, Terminated: False
Action: [0.53862214 1.2345076 ], New Position: [1. 1.], New Velocity: [0 0], Reward: -1, Terminated: False
Action: [ 0.3613651  -0.18476215], New Position: [1. 1.], New Velocity: [0 0], Reward: -1, Terminated: False
Action: [1.1201079 1.5627996], New Position: [1. 1.], New Velocity: [0 0], Reward: -1, Terminated: False
Action: [-0.22943336 -1.0974171 ], New Position: [1. 1.], New Velocity: [0 0], Reward: -1, Terminated: False
Action: [-1.6668606  1.9366153], New Position: [1. 1.], New Velocity: [0 0], Reward: -1, Terminated: False
Action: [-0.86026174 -0.25805628], New Position: [1. 1.], N

## **Case 4: Continuous Markov Decision Process with Probabilistic Reward**

### **Environment Definition**

In [12]:
class ContProbEnv(gym.Env):
    def __init__(self, layout, drone_location, target_location, initial_velocity = np.array([0, 0]), completion_reward = 100000, 
                 obstacle_hit_reward = -100000, reduce_dist_reward = 10, increase_dist_reward = -5, stand_still_reward = -1, speed_limit_x = 5,
                 speed_limit_y = 5, acceleration_limit_x = 2, acceleration_limit_y = 2, time_step = 0.5, distance_threshold = 0.1,
                 wind_acceleration_x_bound = (-1, 1), wind_acceleration_y_bound = (-1, 1)):
        
        super(ContProbEnv, self).__init__()  # Ensure compatibility by calling the parent constructor

        self.layout = layout # Layout is made an attribute of the object so that it can be used in the methods
        self.layout_duplicate = layout.copy() # Duplicate layout to prevent overwriting
        self.rows, self.columns = np.shape(layout) # Obtain the number of rows and columns
        self.raster_dict = raster_scan(layout) # Obtain the dictionary with the raster indexation
        self.start = drone_location # Starting position of the drone
        self.target_location = np.array(target_location) # Target/ Delivery location
        self.agent_location = np.array([self.start[0], self.start[1]]) # Initialize the agent location
        # self.step_count = 0 # Initialize the step count (optional)
        self.velocity = initial_velocity # Initialize the velocity of the agent
        self.speed_limit_x = speed_limit_x # Speed limit in the x-direction
        self.speed_limit_y = speed_limit_y # Speed limit in the y-direction
        self.time_step = time_step # Time step for the simulation
        self.distance_threshold = distance_threshold # Distance threshold for the agent to reach the target
        self.wind_acceleration_x_bound = wind_acceleration_x_bound # Wind acceleration bound in the x-direction
        self.wind_acceleration_y_bound = wind_acceleration_y_bound # Wind acceleration bound in the y-direction

        #######################################################################################################################
        
        # Rewards

        self.completion_reward = completion_reward
        self.obstacle_hit_reward = obstacle_hit_reward
        self.reduce_dist_reward = reduce_dist_reward
        self.increase_dist_reward = increase_dist_reward
        self.reduce_dist_reward = reduce_dist_reward
        self.stand_still_reward = stand_still_reward

        #######################################################################################################################

        # Observations are dictionaries with the agent's location. Note that the target's location is fixed.

        """
        
        We prevent the drone from running over obstacles with the reward mechanism.
        
        """

        self.observation_space = spaces.Dict({
            "agent_position": spaces.Box(low=np.array([0, 0]), high=np.array([self.rows, self.columns]), dtype = np.float32),
            "agent_velocity": spaces.Box(low = np.array([-self.speed_limit_x, -self.speed_limit_y]), 
                                   high = np.array([self.speed_limit_x, self.speed_limit_y]), dtype = np.float32)
        })

        #######################################################################################################################

        self.action_space = spaces.Box(low = np.array([-acceleration_limit_x, -acceleration_limit_y]),
                                        high = np.array([acceleration_limit_x, acceleration_limit_y]), dtype = np.float32)

    ###########################################################################################################################    

    def reset(self, seed = None, **kwargs):
    
        super().reset(seed = seed) # Set the seed for reproducibility
        if seed is not None:
            np.random.seed(seed)

        self.agent_location = self.start # Reset agent location
        self.velocity = np.array([0, 0]) # Reset the velocity
        self.layout = self.layout_duplicate.copy() # Reset the layout

        # Return the observation and an empty dictionary as additional info (as required by Gym)
        return {'agent_position': np.array(self.agent_location), 'agent_velocity': np.array(self.velocity)}, {}
    
    ###########################################################################################################################

    def step(self, action):

        position = self.agent_location
        velocity = self.velocity
        acceleration_x, acceleration_y = action
        
        # Update velocity and position
        velocity[0] = velocity[0] + (acceleration_x * self.time_step)
        velocity[1] = velocity[1] + (acceleration_y * self.time_step)
        position_new = position + (velocity * self.time_step)

        # Check if there are obstacles within the distance_threshold from the new position
        
        possible_obstacle_coordinates = {
                                            "right_up": (np.ceil(position_new[0]), np.ceil(position_new[1])),
                                            "right_down": (np.ceil(position_new[0]), np.floor(position_new[1])),
                                            "left_up": (np.floor(position_new[0]), np.ceil(position_new[1])),
                                            "left_down": (np.floor(position_new[0]), np.floor(position_new[1]))
                                        }

        for value in possible_obstacle_coordinates.values():
            if value[0] >= self.rows or value[1] >= self.columns or value[0] < 0 or value[1] < 0:
                reward = self.obstacle_hit_reward
                terminated = False
                truncated = False
                observation = {'agent_position': position, 'agent_velocity': velocity}
                info = {}  # Add any relevant information here (e.g., additional metrics)
                return observation, reward, terminated, truncated, info
            if self.layout[int(value[0]), int(value[1])] == '#':
                coordinate = np.array([value[0], value[1]])
                if np.linalg.norm(position_new - coordinate) < self.distance_threshold:
                    reward = self.obstacle_hit_reward
                    terminated = False
                    truncated = False
                    observation = {'agent_position': position, 'agent_velocity': velocity}
                    info = {}  # Add any relevant information here (e.g., additional metrics)
                    return observation, reward, terminated, truncated, info

        # Update the agent's position and velocity

        self.agent_location = position_new
        self.velocity = velocity
        if np.linalg.norm(self.agent_location - self.target_location) < np.linalg.norm(position - self.target_location):
            reward = self.reduce_dist_reward + np.random.uniform(self.wind_acceleration_x_bound[0], self.wind_acceleration_x_bound[1]) + np.random.uniform(self.wind_acceleration_y_bound[0], self.wind_acceleration_y_bound[1])
            terminated = False
            truncated = False
            observation = {'agent_position': position_new, 'agent_velocity': velocity}
            info = {}
            return observation, reward, terminated, truncated, info
        
        if np.linalg.norm(self.agent_location - self.target_location) > np.linalg.norm(position - self.target_location):
            reward = self.increase_dist_reward + np.random.uniform(self.wind_acceleration_x_bound[0], self.wind_acceleration_x_bound[1]) + np.random.uniform(self.wind_acceleration_y_bound[0], self.wind_acceleration_y_bound[1])
            terminated = False
            truncated = False
            observation = {'agent_position': position_new, 'agent_velocity': velocity}
            info = {}
            return observation, reward, terminated, truncated, info

        # An episode is done if the agent is within the distance threshold from the target

        if np.linalg.norm(self.agent_location - self.target_location) < self.distance_threshold:
            terminated = True
            truncated = False
            reward = self.completion_reward
            observation = {'agent_position': np.array(self.agent_location), 'agent_velocity': self.velocity}
            info = {}  # Add any relevant information here (e.g., additional metrics)
            return observation, reward, terminated, truncated, info
        
        terminated = False
        truncated = False
        reward = self.stand_still_reward
        observation = {'agent_position': np.array(self.agent_location), 'agent_velocity': self.velocity}
        info = {}  # Add any relevant information here (e.g., additional metrics)
        return observation, reward, terminated, truncated, info

    ###########################################################################################################################

    def render(self, mode = 'human'):
        print(self.layout)  # Print the layout with the agent's current position
        print("Velocity of the Agent: ", self.velocity) # Print the velocity of the agent
        print("Position of the Agent: ", self.agent_location) # Print the agent's location
        return


### **Training Phase**

In [86]:
print("Initial Setup")
layout_size = (8, 8)
drone_location = (1, 1)
target_location = (6, 6)
layout_RL_Learning = strictly_solvable(layout_size, drone_location, target_location, obstacle_density = 0.3) # Generate a solvable layout
env = ContProbEnv(layout_RL_Learning, drone_location, target_location) # Load the environment
print(env.layout)

# Instantiate the model with improved hyperparameters
model = PPO('MultiInputPolicy', env, verbose = 1)

model.learn(total_timesteps = 10000, log_interval = 10) # Train the model with more timesteps and log interval for progress

Initial Setup
Iterations take:  0
[['#' '#' '#' '#' '#' '#' '#' '#']
 ['#' 'D' ' ' ' ' ' ' ' ' ' ' '#']
 ['#' '#' '#' '#' ' ' ' ' '#' '#']
 ['#' ' ' ' ' '#' ' ' ' ' ' ' '#']
 ['#' ' ' ' ' '#' ' ' '#' ' ' '#']
 ['#' ' ' '#' ' ' ' ' ' ' ' ' '#']
 ['#' '#' '#' ' ' ' ' '#' 'T' '#']
 ['#' '#' '#' '#' '#' '#' '#' '#']]
[['#' '#' '#' '#' '#' '#' '#' '#']
 ['#' 'D' ' ' ' ' ' ' ' ' ' ' '#']
 ['#' '#' '#' '#' ' ' ' ' '#' '#']
 ['#' ' ' ' ' '#' ' ' ' ' ' ' '#']
 ['#' ' ' ' ' '#' ' ' '#' ' ' '#']
 ['#' ' ' '#' ' ' ' ' ' ' ' ' '#']
 ['#' '#' '#' ' ' ' ' '#' 'T' '#']
 ['#' '#' '#' '#' '#' '#' '#' '#']]
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


### **Testing Phase**

In [87]:
print("Initial Setup")
print(layout_RL_Learning)

# Test the model here

obs = env.reset()
counter = 0
for _ in range(10000):
    action, _states = model.predict(obs[0])
    observation, reward, terminated, truncated, info = env.step(action)
    print(f"Action: {action}, New Position: {observation['agent_position']}, New Velocity: {observation['agent_velocity']}, Reward: {reward}, Terminated: {terminated}")
    # env.render()
    counter = counter + 1
    if terminated:
        break

print("Number of steps taken: ", counter)

Initial Setup
[['#' '#' '#' '#' '#' '#' '#' '#']
 ['#' 'D' ' ' ' ' ' ' ' ' ' ' '#']
 ['#' '#' '#' '#' ' ' ' ' '#' '#']
 ['#' ' ' ' ' '#' ' ' ' ' ' ' '#']
 ['#' ' ' ' ' '#' ' ' '#' ' ' '#']
 ['#' ' ' '#' ' ' ' ' ' ' ' ' '#']
 ['#' '#' '#' ' ' ' ' '#' 'T' '#']
 ['#' '#' '#' '#' '#' '#' '#' '#']]
Action: [ 0.63149816 -1.549459  ], New Position: [1. 1.], New Velocity: [0 0], Reward: -1, Terminated: False
Action: [-1.0190611 -0.2758514], New Position: [1. 1.], New Velocity: [0 0], Reward: -1, Terminated: False
Action: [ 0.56908673 -0.7445599 ], New Position: [1. 1.], New Velocity: [0 0], Reward: -1, Terminated: False
Action: [-1.6053652  1.640103 ], New Position: [1. 1.], New Velocity: [0 0], Reward: -1, Terminated: False
Action: [-0.07669318  2.        ], New Position: [1.  1.5], New Velocity: [0 1], Reward: 9.3875442949861, Terminated: False
Action: [-0.3655585 -1.0874821], New Position: [1.  1.5], New Velocity: [0 0], Reward: -1, Terminated: False
Action: [-0.63557553  1.538104  ], New P

# **Algorithm Contrast and Comparison**

## **Discrete Environment with Deterministic Reward**

### **Greedy Algorithm Performance:**

In [15]:
size = [5, 6, 7, 8] # Size of the layout
obstacle_density = [0, 0.1, 0.2, 0.3, 0.4] # Obstacle density
n_test_cases = 51 # Number of test cases
drone_location = (1, 1) # Example drone location
iterations = np.zeros(len(test_cases) * n_test_cases) # Initialize an array to store the number of steps taken
max_iter = 100 # Maximum number of iterations
average_iterations = np.zeros(len(test_cases)) # Initialize an array to store the average number of steps taken

for i in range(len(test_cases)):
    target_location = (size[i // 5] - 2, size[i // 5] - 2)
    for j in range(n_test_cases):
        env = DiscDetEnv(test_cases[i][j], drone_location, target_location)
        state = env.reset()
        terminated = False
        counter = 0

        while not terminated:
            reward, terminated, action = env.step()
            state = env.agent_location
            counter = counter + 1

            if counter > max_iter:
                break

        iterations[i * n_test_cases + j] = counter

for i in range(len(test_cases)):
    average_iterations[i] = np.mean(iterations[i * n_test_cases: (i + 1) * n_test_cases])

for i in range(len(size)):
    for j in range(len(obstacle_density)):
        print(f"Size: {size[i]}, Obstacle Density: {obstacle_density[j]}, Average Iterations: {average_iterations[i * len(obstacle_density) + j]}")

Size: 5, Obstacle Density: 0, Average Iterations: 4.0
Size: 5, Obstacle Density: 0.1, Average Iterations: 6.019607843137255
Size: 5, Obstacle Density: 0.2, Average Iterations: 4.235294117647059
Size: 5, Obstacle Density: 0.3, Average Iterations: 10.450980392156863
Size: 5, Obstacle Density: 0.4, Average Iterations: 6.098039215686274
Size: 6, Obstacle Density: 0, Average Iterations: 6.0
Size: 6, Obstacle Density: 0.1, Average Iterations: 8.490196078431373
Size: 6, Obstacle Density: 0.2, Average Iterations: 9.0
Size: 6, Obstacle Density: 0.3, Average Iterations: 13.27450980392157
Size: 6, Obstacle Density: 0.4, Average Iterations: 9.156862745098039
Size: 7, Obstacle Density: 0, Average Iterations: 8.0
Size: 7, Obstacle Density: 0.1, Average Iterations: 10.450980392156863
Size: 7, Obstacle Density: 0.2, Average Iterations: 13.294117647058824
Size: 7, Obstacle Density: 0.3, Average Iterations: 21.294117647058822
Size: 7, Obstacle Density: 0.4, Average Iterations: 15.764705882352942
Size: 8

### **Header Code**

In [ ]:
size = [5, 6, 7, 8] # Size of the layout
obstacle_density = [0, 0.1, 0.2, 0.3, 0.4] # Obstacle density
n_test_cases = 51 # Number of test cases
drone_location = (1, 1) # Example drone location
iterations = np.zeros(len(test_cases) * n_test_cases) # Initialize an array to store the number of steps taken
max_iter = 1000 # Maximum number of iterations
average_iterations = np.zeros(len(test_cases)) # Initialize an array to store the average number of steps taken
counter = 0
base_timesteps = 2500

### **DQN Performance**

In [ ]:
for i in range(len(test_cases)):
    target_location = (size[i // 5] - 2, size[i // 5] - 2)
    if (i + 1) % 5 == 0:
        env = DiscDetCompatEnv(test_cases[i][0], drone_location, target_location)

        # Instantiate the model with improved hyperparameters
        model = DQN('MultiInputPolicy', env, 
                    verbose=1,
                    learning_rate = 1e-2,    # Slightly lower learning rate
                    buffer_size = 100000,    # Increase buffer size for more experience replay
                    batch_size = 32,         # Batch size for more stable training
                    learning_starts = 100,  # Start learning after 100 timesteps
                    exploration_fraction = 0.5,  # Keep exploration high initially
                    exploration_final_eps = 0.25,  # Final epsilon after decay
                    gamma = 0.35)  # Discount factor (gamma) for the future reward

        model.learn(total_timesteps = base_timesteps * i, log_interval = 4) # Train the model with more timesteps and log interval for progress

        for j in range(i - 4, i + 1, 1):
            for k in range(n_test_cases):
                env = DiscDetCompatEnv(test_cases[j][k], drone_location, target_location)
                obs = env.reset()
                terminated = False
                counter = 0

                for _ in range(max_iter):
                    action, _states = model.predict(obs[0])
                    observation, reward, terminated, truncated, info = env.step(action)
                    counter = counter + 1
                    if terminated:
                        break

                iterations[j * n_test_cases + k] = counter

for i in range(len(test_cases)):
    average_iterations[i] = np.mean(iterations[i * n_test_cases: (i + 1) * n_test_cases])

for i in range(len(size)):
    for j in range(len(obstacle_density)):
        print(f"Size: {size[i]}, Obstacle Density: {obstacle_density[j]}, Average Iterations: {average_iterations[i * len(obstacle_density) + j]}")

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
-----------------------------------
| rollout/            |           |
|    ep_len_mean      | 60.2      |
|    ep_rew_mean      | -2.92e+06 |
|    exploration_rate | 0.25      |
| time/               |           |
|    episodes         | 4         |
|    fps              | 975       |
|    time_elapsed     | 0         |
|    total_timesteps  | 241       |
| train/              |           |
|    learning_rate    | 0.01      |
|    loss             | 6.56e+04  |
|    n_updates        | 35        |
-----------------------------------
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
-----------------------------------
| rollout/            |           |
|    ep_len_mean      | 52.8      |
|    ep_rew_mean      | -2.72e+06 |
|    exploration_rate | 0.548     

### **PPO Performance**

In [ ]:
for i in range(len(test_cases)):
    target_location = (size[i // 5] - 2, size[i // 5] - 2)
    if (i + 1) % 5 == 0:
        env = DiscDetCompatEnv(test_cases[i][0], drone_location, target_location)

        # Instantiate the model with improved hyperparameters
        model = PPO('MultiInputPolicy', env, verbose=1)

        model.learn(total_timesteps = base_timesteps * i, log_interval = 4) # Train the model with more timesteps and log interval for progress

        for j in range(i - 4, i + 1, 1):
            for k in range(n_test_cases):
                env = DiscDetCompatEnv(test_cases[j][k], drone_location, target_location)
                obs = env.reset()
                terminated = False
                counter = 0

                for _ in range(max_iter):
                    action, _states = model.predict(obs[0])
                    observation, reward, terminated, truncated, info = env.step(action)
                    counter = counter + 1
                    if terminated:
                        break

                iterations[j * n_test_cases + k] = counter

for i in range(len(test_cases)):
    average_iterations[i] = np.mean(iterations[i * n_test_cases: (i + 1) * n_test_cases])

for i in range(len(size)):
    for j in range(len(obstacle_density)):
        print(f"Size: {size[i]}, Obstacle Density: {obstacle_density[j]}, Average Iterations: {average_iterations[i * len(obstacle_density) + j]}")

### **A2C Performance**

In [ ]:
for i in range(len(test_cases)):
    target_location = (size[i // 5] - 2, size[i // 5] - 2)
    if (i + 1) % 5 == 0:
        env = DiscDetCompatEnv(test_cases[i][0], drone_location, target_location)

        # Instantiate the model with improved hyperparameters
        model = A2C('MultiInputPolicy', env, verbose=1)

        model.learn(total_timesteps = base_timesteps * i, log_interval = 4) # Train the model with more timesteps and log interval for progress

        for j in range(i - 4, i + 1, 1):
            for k in range(n_test_cases):
                env = DiscDetCompatEnv(test_cases[j][k], drone_location, target_location)
                obs = env.reset()
                terminated = False
                counter = 0

                for _ in range(max_iter):
                    action, _states = model.predict(obs[0])
                    observation, reward, terminated, truncated, info = env.step(action)
                    counter = counter + 1
                    if terminated:
                        break

                iterations[j * n_test_cases + k] = counter

for i in range(len(test_cases)):
    average_iterations[i] = np.mean(iterations[i * n_test_cases: (i + 1) * n_test_cases])

for i in range(len(size)):
    for j in range(len(obstacle_density)):
        print(f"Size: {size[i]}, Obstacle Density: {obstacle_density[j]}, Average Iterations: {average_iterations[i * len(obstacle_density) + j]}")

## **Discrete Environment with Probabilistic Reward**

### **Header Code**

In [ ]:
size = [5, 6, 7, 8] # Size of the layout
obstacle_density = [0, 0.1, 0.2, 0.3, 0.4] # Obstacle density
n_test_cases = 51 # Number of test cases
drone_location = (1, 1) # Example drone location
iterations = np.zeros(len(test_cases) * n_test_cases) # Initialize an array to store the number of steps taken
max_iter = 1000 # Maximum number of iterations
average_iterations = np.zeros(len(test_cases)) # Initialize an array to store the average number of steps taken
counter = 0
base_timesteps = 2500

### **DQN Performance**

In [ ]:
for i in range(len(test_cases)):
    target_location = (size[i // 5] - 2, size[i // 5] - 2)
    if (i + 1) % 5 == 0:
        env = DiscProbEnv(test_cases[i][0], drone_location, target_location)

        # Instantiate the model with improved hyperparameters
        model = DQN('MultiInputPolicy', env, verbose=1)

        model.learn(total_timesteps = base_timesteps * i, log_interval = 4) # Train the model with more timesteps and log interval for progress

        for j in range(i - 4, i + 1, 1):
            for k in range(n_test_cases):
                env = DiscProbEnv(test_cases[j][k], drone_location, target_location)
                obs = env.reset()
                terminated = False
                counter = 0

                for _ in range(max_iter):
                    action, _states = model.predict(obs[0])
                    observation, reward, terminated, truncated, info = env.step(action)
                    counter = counter + 1
                    if terminated:
                        break

                iterations[j * n_test_cases + k] = counter

for i in range(len(test_cases)):
    average_iterations[i] = np.mean(iterations[i * n_test_cases: (i + 1) * n_test_cases])

for i in range(len(size)):
    for j in range(len(obstacle_density)):
        print(f"Size: {size[i]}, Obstacle Density: {obstacle_density[j]}, Average Iterations: {average_iterations[i * len(obstacle_density) + j]}")

### **PPO Performance**

In [ ]:
for i in range(len(test_cases)):
    target_location = (size[i // 5] - 2, size[i // 5] - 2)
    if (i + 1) % 5 == 0:
        env = DiscProbEnv(test_cases[i][0], drone_location, target_location)

        # Instantiate the model with improved hyperparameters
        model = PPO('MultiInputPolicy', env, verbose = 1)

        model.learn(total_timesteps = base_timesteps * i, log_interval = 4) # Train the model with more timesteps and log interval for progress

        for j in range(i - 4, i + 1, 1):
            for k in range(n_test_cases):
                env = DiscProbEnv(test_cases[j][k], drone_location, target_location)
                obs = env.reset()
                terminated = False
                counter = 0

                for _ in range(max_iter):
                    action, _states = model.predict(obs[0])
                    observation, reward, terminated, truncated, info = env.step(action)
                    counter = counter + 1
                    if terminated:
                        break

                iterations[j * n_test_cases + k] = counter

for i in range(len(test_cases)):
    average_iterations[i] = np.mean(iterations[i * n_test_cases: (i + 1) * n_test_cases])

for i in range(len(size)):
    for j in range(len(obstacle_density)):
        print(f"Size: {size[i]}, Obstacle Density: {obstacle_density[j]}, Average Iterations: {average_iterations[i * len(obstacle_density) + j]}")

### **A2C Performance**

In [ ]:
for i in range(len(test_cases)):
    target_location = (size[i // 5] - 2, size[i // 5] - 2)
    if (i + 1) % 5 == 0:
        env = DiscProbEnv(test_cases[i][0], drone_location, target_location)

        # Instantiate the model with improved hyperparameters
        model = A2C('MultiInputPolicy', env, verbose=1)

        model.learn(total_timesteps = base_timesteps * i, log_interval = 4) # Train the model with more timesteps and log interval for progress

        for j in range(i - 4, i + 1, 1):
            for k in range(n_test_cases):
                env = DiscProbEnv(test_cases[j][k], drone_location, target_location)
                obs = env.reset()
                terminated = False
                counter = 0

                for _ in range(max_iter):
                    action, _states = model.predict(obs[0])
                    observation, reward, terminated, truncated, info = env.step(action)
                    counter = counter + 1
                    if terminated:
                        break

                iterations[j * n_test_cases + k] = counter

for i in range(len(test_cases)):
    average_iterations[i] = np.mean(iterations[i * n_test_cases: (i + 1) * n_test_cases])

for i in range(len(size)):
    for j in range(len(obstacle_density)):
        print(f"Size: {size[i]}, Obstacle Density: {obstacle_density[j]}, Average Iterations: {average_iterations[i * len(obstacle_density) + j]}")

## **Continuous Environment with Deterministic Reward**

### **Header Code**

In [26]:
size = [5, 6, 7, 8] # Size of the layout
obstacle_density = [0, 0.1, 0.2, 0.3, 0.4] # Obstacle density
n_test_cases = 51 # Number of test cases
drone_location = (1, 1) # Example drone location
iterations = np.zeros(len(test_cases) * n_test_cases) # Initialize an array to store the number of steps taken
max_iter = 9000 # Maximum number of iterations
average_iterations = np.zeros(len(test_cases)) # Initialize an array to store the average number of steps taken
counter = 0
base_timesteps = 1000

### **PPO Performance**

In [17]:
for i in range(len(test_cases)):
    target_location = (size[i // 5] - 2, size[i // 5] - 2)
    if (i + 1) % 5 == 0:
        env = ContDetEnv(test_cases[i][0], drone_location, target_location)

        # Instantiate the model with improved hyperparameters
        model = PPO('MultiInputPolicy', env, verbose = 0)

        model.learn(total_timesteps = base_timesteps * i, log_interval = 4) # Train the model with more timesteps and log interval for progress

        for j in range(i - 4, i + 1, 1):
            for k in range(n_test_cases):
                env = ContDetEnv(test_cases[j][k], drone_location, target_location)
                obs = env.reset()
                terminated = False
                counter = 0

                for _ in range(max_iter):
                    action, _states = model.predict(obs[0])
                    observation, reward, terminated, truncated, info = env.step(action)
                    counter = counter + 1
                    if terminated:
                        break

                iterations[j * n_test_cases + k] = counter

for i in range(len(test_cases)):
    average_iterations[i] = np.mean(iterations[i * n_test_cases: (i + 1) * n_test_cases])

for i in range(len(size)):
    for j in range(len(obstacle_density)):
        print(f"Size: {size[i]}, Obstacle Density: {obstacle_density[j]}, Average Iterations: {average_iterations[i * len(obstacle_density) + j]}")

Size: 5, Obstacle Density: 0, Average Iterations: 2146.6862745098038
Size: 5, Obstacle Density: 0.1, Average Iterations: 2059.7450980392155
Size: 5, Obstacle Density: 0.2, Average Iterations: 2289.5098039215686
Size: 5, Obstacle Density: 0.3, Average Iterations: 2256.274509803922
Size: 5, Obstacle Density: 0.4, Average Iterations: 1676.7843137254902
Size: 6, Obstacle Density: 0, Average Iterations: 3167.7843137254904
Size: 6, Obstacle Density: 0.1, Average Iterations: 2636.4313725490197
Size: 6, Obstacle Density: 0.2, Average Iterations: 3529.274509803922
Size: 6, Obstacle Density: 0.3, Average Iterations: 3117.235294117647
Size: 6, Obstacle Density: 0.4, Average Iterations: 3542.0
Size: 7, Obstacle Density: 0, Average Iterations: 3299.8039215686276
Size: 7, Obstacle Density: 0.1, Average Iterations: 4152.666666666667
Size: 7, Obstacle Density: 0.2, Average Iterations: 4111.745098039216
Size: 7, Obstacle Density: 0.3, Average Iterations: 4237.137254901961
Size: 7, Obstacle Density: 0.4

### **A2C Performance**

In [18]:
for i in range(len(test_cases)):
    target_location = (size[i // 5] - 2, size[i // 5] - 2)
    if (i + 1) % 5 == 0:
        env = ContDetEnv(test_cases[i][0], drone_location, target_location)

        # Instantiate the model with improved hyperparameters
        model = A2C('MultiInputPolicy', env, verbose = 0)

        model.learn(total_timesteps = base_timesteps * i, log_interval = 4) # Train the model with more timesteps and log interval for progress

        for j in range(i - 4, i + 1, 1):
            for k in range(n_test_cases):
                env = ContDetEnv(test_cases[j][k], drone_location, target_location)
                obs = env.reset()
                terminated = False
                counter = 0

                for _ in range(max_iter):
                    action, _states = model.predict(obs[0])
                    observation, reward, terminated, truncated, info = env.step(action)
                    counter = counter + 1
                    if terminated:
                        break

                iterations[j * n_test_cases + k] = counter

for i in range(len(test_cases)):
    average_iterations[i] = np.mean(iterations[i * n_test_cases: (i + 1) * n_test_cases])

for i in range(len(size)):
    for j in range(len(obstacle_density)):
        print(f"Size: {size[i]}, Obstacle Density: {obstacle_density[j]}, Average Iterations: {average_iterations[i * len(obstacle_density) + j]}")

Size: 5, Obstacle Density: 0, Average Iterations: 1948.3529411764705
Size: 5, Obstacle Density: 0.1, Average Iterations: 1740.764705882353
Size: 5, Obstacle Density: 0.2, Average Iterations: 1641.686274509804
Size: 5, Obstacle Density: 0.3, Average Iterations: 1556.4705882352941
Size: 5, Obstacle Density: 0.4, Average Iterations: 1741.4705882352941
Size: 6, Obstacle Density: 0, Average Iterations: 9000.0
Size: 6, Obstacle Density: 0.1, Average Iterations: 9000.0
Size: 6, Obstacle Density: 0.2, Average Iterations: 9000.0
Size: 6, Obstacle Density: 0.3, Average Iterations: 9000.0
Size: 6, Obstacle Density: 0.4, Average Iterations: 9000.0
Size: 7, Obstacle Density: 0, Average Iterations: 9000.0
Size: 7, Obstacle Density: 0.1, Average Iterations: 9000.0
Size: 7, Obstacle Density: 0.2, Average Iterations: 9000.0
Size: 7, Obstacle Density: 0.3, Average Iterations: 9000.0
Size: 7, Obstacle Density: 0.4, Average Iterations: 9000.0
Size: 8, Obstacle Density: 0, Average Iterations: 9000.0
Size: 

### **SAC Performance**

In [27]:
for i in range(len(test_cases)):
    target_location = (size[i // 5] - 2, size[i // 5] - 2)
    if (i + 1) % 5 == 0:
        env = ContDetEnv(test_cases[i][0], drone_location, target_location)

        # Instantiate the model with improved hyperparameters
        model = SAC('MultiInputPolicy', env, verbose = 1)

        model.learn(total_timesteps = base_timesteps * i, log_interval = 4) # Train the model with more timesteps and log interval for progress

        for j in range(i - 4, i + 1, 1):
            for k in range(n_test_cases):
                env = ContDetEnv(test_cases[j][k], drone_location, target_location)
                obs = env.reset()
                terminated = False
                counter = 0

                for _ in range(max_iter):
                    action, _states = model.predict(obs[0])
                    observation, reward, terminated, truncated, info = env.step(action)
                    counter = counter + 1
                    if terminated:
                        break

                iterations[j * n_test_cases + k] = counter

for i in range(len(test_cases)):
    average_iterations[i] = np.mean(iterations[i * n_test_cases: (i + 1) * n_test_cases])

for i in range(len(size)):
    for j in range(len(obstacle_density)):
        print(f"Size: {size[i]}, Obstacle Density: {obstacle_density[j]}, Average Iterations: {average_iterations[i * len(obstacle_density) + j]}")

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Size: 5, Obstacle Density: 0, Average Iterations: 9000.0
Size: 5, Obstacle Density: 0.1, Average Iterations: 9000.0
Size: 5, Obstacle Density: 0.2, Average Iterations: 9000.0
Size: 5, Obstacle Density: 0.3, Average Iterations: 9000.0
Size: 5, Obstacle Density: 0.4, Average Iterations: 9000.0
Size: 6, Obstacle Density: 0, Average Iterations: 9000.0
Size: 6, Obstacle Density: 0.1, Average Iterations: 9000.0
Size: 6, Obstacle Density: 0.2, Average Iterations: 9000.0
Size: 6, Obstacle Density: 0.3, Average Iterations: 9000.0
Size: 6, Obstacle Density: 0.4, Average Iterations: 9000.0
Size: 7, Obstacle Density: 0, Average 

## **Continuous Environment with Probabilistic Reward**

### **Header Code**

In [28]:
size = [5, 6, 7, 8] # Size of the layout
obstacle_density = [0, 0.1, 0.2, 0.3, 0.4] # Obstacle density
n_test_cases = 51 # Number of test cases
drone_location = (1, 1) # Example drone location
iterations = np.zeros(len(test_cases) * n_test_cases) # Initialize an array to store the number of steps taken
max_iter = 9000 # Maximum number of iterations
average_iterations = np.zeros(len(test_cases)) # Initialize an array to store the average number of steps taken
counter = 0
base_timesteps = 1000

### **PPO Performance**

In [ ]:
for i in range(len(test_cases)):
    target_location = (size[i // 5] - 2, size[i // 5] - 2)
    if (i + 1) % 5 == 0:
        env = ContProbEnv(test_cases[i][0], drone_location, target_location)

        # Instantiate the model with improved hyperparameters
        model = PPO('MultiInputPolicy', env, verbose=1)

        model.learn(total_timesteps = base_timesteps * i, log_interval = 4) # Train the model with more timesteps and log interval for progress

        for j in range(i - 4, i + 1, 1):
            for k in range(n_test_cases):
                env = ContProbEnv(test_cases[j][k], drone_location, target_location)
                obs = env.reset()
                terminated = False
                counter = 0

                for _ in range(max_iter):
                    action, _states = model.predict(obs[0])
                    observation, reward, terminated, truncated, info = env.step(action)
                    counter = counter + 1
                    if terminated:
                        break

                iterations[j * n_test_cases + k] = counter

for i in range(len(test_cases)):
    average_iterations[i] = np.mean(iterations[i * n_test_cases: (i + 1) * n_test_cases])

for i in range(len(size)):
    for j in range(len(obstacle_density)):
        print(f"Size: {size[i]}, Obstacle Density: {obstacle_density[j]}, Average Iterations: {average_iterations[i * len(obstacle_density) + j]}")

### **A2C Performance**

In [ ]:
for i in range(len(test_cases)):
    target_location = (size[i // 5] - 2, size[i // 5] - 2)
    if (i + 1) % 5 == 0:
        env = ContProbEnv(test_cases[i][0], drone_location, target_location)

        # Instantiate the model with improved hyperparameters
        model = A2C('MultiInputPolicy', env, verbose=1)

        model.learn(total_timesteps = base_timesteps * i, log_interval = 4) # Train the model with more timesteps and log interval for progress

        for j in range(i - 4, i + 1, 1):
            for k in range(n_test_cases):
                env = ContProbEnv(test_cases[j][k], drone_location, target_location)
                obs = env.reset()
                terminated = False
                counter = 0

                for _ in range(max_iter):
                    action, _states = model.predict(obs[0])
                    observation, reward, terminated, truncated, info = env.step(action)
                    counter = counter + 1
                    if terminated:
                        break

                iterations[j * n_test_cases + k] = counter

for i in range(len(test_cases)):
    average_iterations[i] = np.mean(iterations[i * n_test_cases: (i + 1) * n_test_cases])

for i in range(len(size)):
    for j in range(len(obstacle_density)):
        print(f"Size: {size[i]}, Obstacle Density: {obstacle_density[j]}, Average Iterations: {average_iterations[i * len(obstacle_density) + j]}")

### **SAC Performance**

In [29]:
for i in range(len(test_cases)):
    target_location = (size[i // 5] - 2, size[i // 5] - 2)
    if (i + 1) % 5 == 0:
        env = ContProbEnv(test_cases[i][0], drone_location, target_location)

        # Instantiate the model with improved hyperparameters
        model = SAC('MultiInputPolicy', env, verbose = 1)

        model.learn(total_timesteps = base_timesteps * i, log_interval = 4) # Train the model with more timesteps and log interval for progress

        for j in range(i - 4, i + 1, 1):
            for k in range(n_test_cases):
                env = ContProbEnv(test_cases[j][k], drone_location, target_location)
                obs = env.reset()
                terminated = False
                counter = 0

                for _ in range(max_iter):
                    action, _states = model.predict(obs[0])
                    observation, reward, terminated, truncated, info = env.step(action)
                    counter = counter + 1
                    if terminated:
                        break

                iterations[j * n_test_cases + k] = counter

for i in range(len(test_cases)):
    average_iterations[i] = np.mean(iterations[i * n_test_cases: (i + 1) * n_test_cases])

for i in range(len(size)):
    for j in range(len(obstacle_density)):
        print(f"Size: {size[i]}, Obstacle Density: {obstacle_density[j]}, Average Iterations: {average_iterations[i * len(obstacle_density) + j]}")

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Size: 5, Obstacle Density: 0, Average Iterations: 9000.0
Size: 5, Obstacle Density: 0.1, Average Iterations: 9000.0
Size: 5, Obstacle Density: 0.2, Average Iterations: 9000.0
Size: 5, Obstacle Density: 0.3, Average Iterations: 9000.0
Size: 5, Obstacle Density: 0.4, Average Iterations: 9000.0
Size: 6, Obstacle Density: 0, Average Iterations: 9000.0
Size: 6, Obstacle Density: 0.1, Average Iterations: 9000.0
Size: 6, Obstacle Density: 0.2, Average Iterations: 9000.0
Size: 6, Obstacle Density: 0.3, Average Iterations: 9000.0
Size: 6, Obstacle Density: 0.4, Average Iterations: 9000.0
Size: 7, Obstacle Density: 0, Average 